In [63]:
import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc

import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc
# SKLearn related imports
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
import spacy
import hashlib

from tqdm import tqdm
from collections import Counter
from spacy.matcher import Matcher
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin




In [64]:
# configuration
data_set = pc.getFile()  # gets path of tfile, you may replace with a string

In [65]:
# configuration
label = "label" # TODO

In [66]:
# Load the dataset
df = pd.read_csv(data_set) # never modify

# Show basic info
print(df.dtypes)            # all columns/types
print("Shape: ",df.shape)   # shape
df.head(10)



reviewerID       object
reviewerName     object
reviewText       object
overall           int64
summary          object
reviewTime       object
rates_count       int64
helpful_count     int64
rating            int64
dtype: object
Shape:  (49992, 9)


,reviewerID,reviewerName,reviewText,overall,summary,reviewTime,rates_count,helpful_count,rating
0,A3UPFTGAWZ3G2R,David J. Loftus,"Jenkins, a history professor and Member of Par...",4,"Quite readable, nicely done","12 6, 2001",40,37,4
1,A1XTKTLNSCRLDS,Ellen Rappaport,Detective Inspector Erlendur Sveinsson is at h...,5,Mesmerizing in depth,"02 23, 2014",0,0,5
2,A1A77B6DQQH436,"crescamp ""esc""",I didn't read this. I purchased it for a gift...,3,10-minute life lessons for kids,"02 12, 2013",3,0,3
3,AEAF4MRYHJZI,"Angelia Menchan ""acvermen.blogspot.com""",Fierce Angels by Sheri Park reads like a disse...,4,So FIERCE,"03 24, 2010",9,9,4
4,A3B7KU72LGWFER,"Grifel ""Tea Time""",Clearly this author had two goals in mind: 1) ...,1,Drivel!,"06 21, 2003",19,13,1
5,A3JD07VHDLT5FF,"isala ""Isabel and Lars""",This is a collection of stories and memories b...,5,Compelling stories by ordinary people,"03 19, 2005",7,5,5
6,A1JBVAMC9YU0WD,ED,I am so glad I borrowed this 'book' from the l...,1,this 'cheese' stinks,"09 25, 2000",6,5,1
7,A1O7OXC13G9X3E,"kayp75 ""kayp75""","Really enjoyable, couldn't put it down. The ch...",5,great read,"10 21, 2013",0,0,5
8,A1RAUVCWYHTQI4,A. Ross,I don't read a lot of science-fiction (maybe f...,3,Lots of Good Elements Fail to Add Up to an Eng...,"08 4, 2010",3,2,3
9,A3PZTH1DTX6O6B,"H. F. Gibbard ""History Buff""",Legendary pornographer Larry Flynt teamed up w...,4,fun but occasionally debatable history of pres...,"06 11, 2011",5,4,4


In [67]:
# Create labels

# creates a new column helpful_count/rates_count
class TransformLabels(TransformerMixin):
    def __init__(self, threshold = .5):
        self.helpful_count = 'helpful_count'
        self.rates_count = 'rates_count'
        self.threshold = threshold
        
    def isHelpful(self, helpful_count, rates_count):
        if rates_count == 0:
            return 0 # we don't know, not helpfull for now
        if helpful_count > rates_count:
            return 5 # five means bad, needs to be droped
        val = helpful_count/rates_count
        if val >= self.threshold:
            return 1
        return 0

    def transform(self, X, *_):
        _X = X.copy()

        label_array = []
        for h, r in zip(_X[self.helpful_count], _X[self.rates_count]):
            is_helpul = self.isHelpful(h,r)
            label_array += [is_helpul]

        out = pd.Series(label_array)
        return out
    
    def fit(self, *_):
        return self

class TextTransformer(TransformerMixin):
    def __init__(self, key):
        self.key = key

    def transform(self, X, *_):
        _X = X.copy()
        words = _X[self.key]
        features =  TfidfTransformer().fit_transform(words)
        return features
    
    def fit(self, *_):
        return self

class LazyTranformer(TransformerMixin):
    def __init__(self, key):
        self.key = key

    def transform(self, X, *_):
        _X = X.copy()
        return _X[self.key].to_frame()
    
    def fit(self, *_):
        return self

class Selector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    

class TextSelector(Selector):
    def transform(self, X):
        return X[self.key]

In [68]:
# Add label to df (without this it's not possible to train the model)
df['label'] = TransformLabels(threshold=.5).fit_transform(df)  
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [69]:
# Show label distribution
pc.showColumnDistribution(df,label)

Number of entries:  49992

Distribution of the column:


0    27871
1    22121
Name: label, dtype: int64


Nan Values:  0


In [70]:
# Define features here, keep standard, concatenate _feature in the end
review_text_feature = Pipeline([
                                ('text_selector', TextSelector("reviewText")),
                                ('tfidf', TfidfVectorizer())
                            ])

# Features
features = FeatureUnion([
                        ('reviewText', review_text_feature)
                        ])

In [71]:
# Pipeline w/ model
model_pipeline = Pipeline([
                        ('features', features),
                        ('classifier', RandomForestClassifier(random_state = 42))
                    ])

In [81]:
# Baseline and print silly values
# returns model
def baseline(train_set, test_set, model):
  # Train the classifier
  model.fit(train_set, train_set[label])

  # Predict
  model_predictions = model.predict(test_set)

  # Print F1 stuff
  print(classification_report(test_set[label], model_predictions))

  _acc = np.mean(model_predictions == test_set[label])
  print('Accuracy: ' + str(_acc))

  return model

def stupidFunction(x):
    if x == 0:
        return "false"
    return "true"
def predictFromFileAndOutPutCSV(model, path):
    data_set_to_predict = pc.getFile()
    df = pd.read_csv(data_set_to_predict) # never modify
    predictions = model.predict(df)
    out = pd.DataFrame({'is_helpful': predictions})
    out['is_helpful'] =  out['is_helpful'].apply(lambda x: stupidFunction(x))
    out.to_csv(path, index=False)


In [79]:
# Get model
model = baseline(df_train, df_test, model_pipeline)

              precision    recall  f1-score   support

           0       0.69      0.78      0.73      5590
           1       0.67      0.57      0.61      4409

    accuracy                           0.68      9999
   macro avg       0.68      0.67      0.67      9999
weighted avg       0.68      0.68      0.68      9999

Accuracy: 0.6830683068306831


In [82]:
predictFromFileAndOutPutCSV(model, 'done.csv')

In [ ]:

# Predict
_predicted = model.predict(_test_data)
_acc = np.mean(_predicted == _test_data[label])
classification_report(_test_data[label], _predicted)
print('Accuracy: ' + str(_acc))